In [13]:
import numpy as np
import pandas as pd

In [8]:
predictions = np.load('../madar_city.npz')['arr_0']


In [11]:
predictions.shape[0], len(predictions[0])


(5200, 113)

In [15]:
madar_folder =  '../../hierarchical-did/data_processed_second/madar_shared_task1/'
test_file = f'{madar_folder}MADAR-Corpus-26-test.lines'
df = pd.read_csv(test_file, sep='\t', header=0)


In [16]:
df

,original_sentence,dialect_city_id,dialect_province_id,dialect_country_id,dialect_region_id
0,لا أعرف كثيراً عن النبيذ ؟ ماذا يناسب هذا الطبق ؟,msa,msa,msa,msa
1,رايح عالمدرسة هون ؟,damascus,damascus,sy,levant
2,قهوه مع كريمة و سكر ، لوسمحت .,sanaa,sanaa,ye,gulf_aden
3,بأي محطة لازم أنزل عشان أروح على امباير ستيت ب...,amman,amman,jo,levant
4,اسمي ميتشيكو تاناكا ، ورقم الرحلة خمسة صفر واح...,jeddah,makkah,sa,gulf
...,...,...,...,...,...
5195,تسعه وعشرين ثمانيه .,benghazi,benghazi,ly,maghreb
5196,اكو اكو واحد متوفر ، تكدر تحجزلي ميز يم الشباج ؟,basra,basra,iq,gulf
5197,اهلا ، روبتك بالحق مزيانة .,tunis,tunis,tn,maghreb
5198,الى طوكيو ، لو سمحت .,muscat,muscat,om,gulf


In [17]:
predictions[0]

array([ 3.0281839e+00, -1.2136688e+00, -1.0954336e+00, -3.6960956e-01,
        2.7500018e-01, -1.6975091e+00, -4.3090153e+00, -8.2746863e-01,
       -1.1228651e-02, -2.2352116e+00, -2.2868514e+00, -1.7479193e-01,
       -1.1477091e+00,  1.4767222e+00,  2.3019087e-02,  2.1527078e+00,
       -1.6219802e+00, -9.8533249e-01, -4.9744576e-01, -9.3938106e-01,
        1.1929229e-01,  5.1029110e-01, -2.4340014e-01, -1.0125523e+00,
       -1.0203371e+00, -1.7168608e+00,  2.0929289e+00, -8.4939325e-01,
        6.8239290e-01,  5.3415388e-01, -1.2658944e+00, -3.5684565e-01,
       -2.6458793e+00, -7.7713799e-01, -2.2621903e+00, -1.2882674e+00,
        3.1715073e-02, -4.3918306e-01, -3.3190916e+00,  1.5457227e+00,
        7.6336306e-01, -8.9144409e-01, -1.1761220e+00, -2.7556467e+00,
       -6.6303074e-01,  4.6812087e-02, -9.9481159e-01, -4.6477919e+00,
       -3.7024903e-01, -3.3576498e+00, -2.2406344e+00, -7.7027559e-02,
       -2.0571427e+00, -3.3632686e+00, -2.9817176e+00, -1.5743995e+00,
      

In [21]:
#Distances
def get_dictionary_locations(filename='../labels/final_label_space.tsv'):
    df = pd.read_csv(filename, sep='\t', header=0)
    map_locator_name = {}
    for index, row in df.iterrows():
        label = ''
        if row['dialect_city_id'] != 'msa' and row['dialect_city_id'] != 'jerusalem' and row['dialect_city_id'] != 'al_suwayda':
            label = f'{row["dialect_city_id"].replace("_", " ")} {row["dialect_country_id"]}'
        elif row['dialect_city_id'] == 'jerusalem':
            label = 'jerusalem'
        elif row['dialect_city_id'] == 'al_suwayda':
            label = 'as suwayda sy'
        if label != '':
            map_locator_name[label] = [row['dialect_city_id'], row['dialect_country_id'], row['dialect_region_id']]
    return map_locator_name
get_dictionary_locations()

{'abu dhabi ae': ['abu_dhabi', 'ae', 'gulf'],
 'dubai ae': ['dubai', 'ae', 'gulf'],
 'fujairah ae': ['fujairah', 'ae', 'gulf'],
 'ras al khaimah ae': ['ras_al_khaimah', 'ae', 'gulf'],
 'umm al quwain ae': ['umm_al_quwain', 'ae', 'gulf'],
 'manama bh': ['manama', 'bh', 'gulf'],
 'hawalli kw': ['hawalli', 'kw', 'gulf'],
 'jahra kw': ['jahra', 'kw', 'gulf'],
 'nizwa om': ['nizwa', 'om', 'gulf'],
 'sohar om': ['sohar', 'om', 'gulf'],
 'sur om': ['sur', 'om', 'gulf'],
 'salalah om': ['salalah', 'om', 'gulf'],
 'khasab om': ['khasab', 'om', 'gulf'],
 'muscat om': ['muscat', 'om', 'gulf'],
 'doha qa': ['doha', 'qa', 'gulf'],
 'al rayyan qa': ['al_rayyan', 'qa', 'gulf'],
 'al madinah sa': ['al_madinah', 'sa', 'gulf'],
 'abha sa': ['abha', 'sa', 'gulf'],
 'hail sa': ['hail', 'sa', 'gulf'],
 'jizan sa': ['jizan', 'sa', 'gulf'],
 'najran sa': ['najran', 'sa', 'gulf'],
 'riyadh sa': ['riyadh', 'sa', 'gulf'],
 'tabuk sa': ['tabuk', 'sa', 'gulf'],
 'buraidah sa': ['buraidah', 'sa', 'gulf'],
 'dammam

In [33]:
from transformers import (AutoTokenizer, AdamW, AutoModelForSequenceClassification, AutoConfig,
                          get_linear_schedule_with_warmup)
import sys
sys.path.append('/Users/nurpeiis/Desktop/Capstone/arabic-did')
import data_utils

In [23]:
model = AutoModelForSequenceClassification.from_pretrained('../models/best_model.pt')


Some weights of the model checkpoint at ../models/best_model.pt were not used when initializing BertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
bert = model.bert
bert.save_pretrained('bert.pt')

In [35]:
city_labels =  '../labels/madar_city_label_id.txt'
labels, label2id, id2label = data_utils.get_label_space(city_labels)

In [38]:
config = AutoConfig.from_pretrained(
        'bert.pt',
        num_labels=len(labels),
        label2id=label2id,
        id2label=id2label,
        finetuning_task='arabic_did'
    )

In [41]:
model_new = AutoModelForSequenceClassification.from_pretrained(
        'bert.pt',
        config=config
    )
model_new

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert.pt and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [42]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element